## Text2SQL with Redshift DB, LLM and Tools
This notebook generates SQL statement and summary of input query based on the given LLM and input question. It also uses tools to get code for gender, race and state of patients.

In [1]:
# !pip install jupyter-black
# !pip install langchain -q
# !pip install langchain-experimental -q
# !pip install boto3 --upgrade -q
# !pip install s3fs --upgrade -q

# !pip install sqlalchemy-redshift psycopg2-binary -q --upgrade
# !pip install sqlalchemy --upgrade -q

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import jupyter_black

jupyter_black.load()

In [4]:
import pandas as pd
import os
import boto3

from tqdm import tqdm

tqdm.pandas()

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

from langchain.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS

from config import *
from utils import *
from function_calls import *

In [11]:
sample_questions = [
    "Find total number of inpatients",
    "select all inpatients in year 2005",
    "count female patients",
    "Find total number of male inpatients.",
    "Find total number of female, black outpatients.",
    "Find total number of female, white patients living in California.",
    "select the outpatients who are diagnosed with diabetes of code list X",
    "select the male inpatients who receive hospitalization because of diabetes of code list X",
    "select the male, white outpatients living in Florida who are diagnosed with disease in code list X and are sent to hospital thereafter",
]

In [5]:
# Get column descriptions in xml format
col_descriptions = get_column_descriptions()
print(col_descriptions[:200])

<data>
  <row>
    <column_name>DESYNPUF_ID</column_name>
    <column_description>Beneficiary Code</column_description>
    <table_name>beneficiary_summary</table_name>
  </row>
  <row>
    <column_na


In [6]:
# Get tables info with sample rows
tables_info = get_db_tables_info()

print(tables_info[:200])

Tables Info already exists! Loading.

CREATE TABLE rwdex_raw_synpuf.beneficiary_summary (
	desynpuf_id VARCHAR(256), 
	bene_birth_dt DATE, 
	bene_death_dt DATE, 
	bene_sex_ident_cd INTEGER, 
	bene_race_cd INTEGER, 
	bene_esrd_ind VARCHAR


In [7]:
# Create LLM for SQL generation
llm = get_bedrock_llm(
    model_id=SQL_MODEL_ID, verbose=True, model_config=SQL_MODEL_CONFIG
)

In [8]:
print(PROMPT)



Human: You are a PostgreSQL expert. Given an input question, first create a syntactically correct PostgreSQL query to run, then look at the results of the query and return the answer to the input question.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table. Database schema is provided in <table-info> tag and input question is in <question> tag. Columns description is also provided within <columns-description> tag. In addition, the database schema name is also added in <schema-name> tag.

Provide only the necessary answer. When count, make sure you count only the distinct ones. When using any table, add the provided schema name in <schema-name> as a prefix.



In [10]:
for question in sample_questions:
    # Find few-shot examples if needed
    few_shot_examples = ""
    if NUM_FEW_SHOT_EXAMPLES:
        # Create embeddings model for few-shot learning
        bedrock_runtime = boto3.client("bedrock-runtime")
        llm_embeddings = BedrockEmbeddings(
            client=bedrock_runtime, model_id=EMBED_MODEL_ID
        )

        # Load golden examples for few shot learning
        with open(FAISS_DICT_EXAMPLES_PATH, "r") as fp:
            # pickle.dump(sample_dict, file)
            golden_examples = json.load(fp)

        # Show some golden examples
        i = 0
        for key, val in golden_examples.items():
            print(f"Question: {key}\nSQL: {val}")
            print("-" * 30)
            i += 1
            if i == 3:
                break

        # Load Vector DB and Create a retriever
        vector_db = FAISS.load_local(FAISS_INDEX_DIR, llm_embeddings)
        retriever = vector_db.as_retriever(search_kwargs={"k": NUM_FEW_SHOT_EXAMPLES})

        few_shot_examples = build_few_shot_examples(
            golden_examples, question, retriever
        )
    print(few_shot_examples)

    inputs = {
        "question": question,
        "db_schema_name": DB_SCHEMA_NAME,
        "table_info": tables_info,
        "columns_description": col_descriptions,
        "query_examples": few_shot_examples,
    }

    prompt = create_prompt(PROMPT, inputs)
    response = run_loop(llm, prompt)

    sql = extract_xml(tags=["sql"], text=response)
    summary = extract_xml(tags=["summary"], text=response)

    print("Input Question:\n", question)
    print("-" * 40)
    print("SQL:\n", sql)
    if sql is None:
        print("Response:\n", response)
        print("-" * 40)
    print("-" * 40)
    print("Question Summary:\n", summary)
    print("=" * 40)


Input Question:
 Find total number of inpatients
----------------------------------------
SQL:
 SELECT COUNT(DISTINCT ic.desynpuf_id) AS num_inpatients
FROM rwdex_raw_synpuf.inpatient_claims ic
----------------------------------------
Question Summary:
 The SQL query counts the distinct beneficiary IDs from the inpatient_claims table to find the total number of unique inpatients.

Input Question:
 select all inpatients in year 2005
----------------------------------------
SQL:
 SELECT DISTINCT ic.desynpuf_id
FROM inpatient_claims ic
WHERE EXTRACT(YEAR FROM ic.clm_from_dt) = 2005
----------------------------------------
Question Summary:
 The SQL query selects distinct beneficiary IDs from the inpatient_claims table where the year extracted from the claim start date is 2005. This will return all unique inpatients that had a claim in the year 2005.

Input Question:
 count female patients
----------------------------------------
SQL:
 SELECT COUNT(DISTINCT desynpuf_id) AS num_patients
FR

### Run for all examples

In [ ]:
# Load input data
df_qa = pd.read_csv(QA_DATA_PATH)

# Remove all gpt4 columns
gpt4_cols = [col for col in df_qa.columns if "gpt4" in col]
df_qa.drop(columns=gpt4_cols, inplace=True)
print(df_qa.shape)
df_qa.head()

In [ ]:
# Count number of executable SQLs
df_qa.query_corrected_with_list_executed.value_counts()

In [ ]:
inputs = {
    "question": "<<question here>>",
    "db_schema_name": DB_SCHEMA_NAME,
    "table_info": tables_info,
    "columns_description": col_descriptions,
    "query_examples": few_shot_examples,
}

gt_sql_executed_col = "query_corrected_with_list_executed"
question_col = "question_with_list"

df_qa0 = df_qa.progress_apply(
    get_llm_output,
    args=(llm, PROMPT, inputs, question_col, gt_sql_executed_col),
    axis=1,
)

In [ ]:
df_qa.head()

In [ ]:
# Display GT and predicted SQL
_ = df_qa.apply(
    print_gt_and_pred_sql,
    args=("question", "query_corrected", "sql_predicted", "question_summary"),
    axis=1,
)

In [ ]:
## Save Outputs
df_qa.to_csv(OUTPUT_PATH, index=False)